### Problem 5.1 - Capacitated transportation problem revisited
*(Marginal costs via dual calculation)*
#' 
Adding necessary packages:

In [ ]:
# using Pkg
# Pkg.add("GLPK")

In [ ]:
## Packages
using JuMP, Cbc, GLPK # We need GLPK for obtaining duals

We start by defining the problem data

In [ ]:
nS = 3 # Number of suppliers
nD = 3 # Number of demand points
nP = 2 # Number of products

S = 1:nS
D = 1:nD
P = 1:nP

A = [] # Set of arcs, we include arcs from all suppliers to all demand points
for s in S, d in D
    push!(A, (s,d))
end

In [ ]:
costs = zeros(nP,nS,nD) # Cost of transporting one unit of product d from supplier s to demand point d 
costs[1,:,:] = [5 5 Inf; 8 9 7; Inf 10 8]
costs[2,:,:] = [Inf 18 Inf; 15 12 14; Inf 20 Inf]

sup = [80 400; 200 1500; 200 300]
dem = [60 300; 100 1000; 200 500]

cap = [Inf 300 0; 300 700 600; 0 Inf Inf];

In [ ]:
## Define model using GLPK
model = Model(GLPK.Optimizer);

## Variables
@variable(model, x[a in A, p in P; costs[p,a[1],a[2]] < Inf] >= 0);

## OF
@objective(model, Min, sum(costs[p,a[1],a[2]]*x[a,p] for p in P, a in A if costs[p,a[1],a[2]] < Inf));

## Constraints
@constraint(model, sup_con[s in S, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[1] == s) <= sup[s,p]);       # sum of everything that leaves supplier s
@constraint(model, dem_con[d in D, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[2] == d) >= dem[d,p]);       # sum of everything that arrives at demand d
@constraint(model, cap_con[a in A; cap[a[1],a[2]] < Inf], sum(x[a,p] for p in P if costs[p,a[1],a[2]] < Inf) <= cap[a[1],a[2]]);# arc capacity constraints

In [ ]:
set_silent(model) # Actually works with GLPK
optimize!(model)
status = termination_status(model)
println(status)

In [ ]:
## Saving the optimal value
obj = objective_value(model)

Function ``dual()`` in the ``JuMP`` library gives the value of the dual variable associated to the constraint at the optimal solution, in other words the *marginal costs*. Here we need using the elemnt-wise operator ``.`` as we have multiple constraints (check the domains). The *marginal costs* value stands for how much adding one unit more to the constraint's RHS (in the case is a $\leq$ constraint) impacts the final result.<br/>
<br/>
One interpretation for the *marginal costs* in this problem is how much the company is willing to pay for expanding the supplies' or the arcs' capacity (depending on if we're analysing the dual of the supplies or the arcs constraints).

In [ ]:
## Computing the duals to infer the marginal costs
mc_supply = dual.(sup_con);
mc_arcs = dual.(cap_con);
for s in S, p in P
    println("The marginal costs for the supply $s for the product $p is: $(mc_supply[s,p])")
end
for a in A
    if cap[a[1],a[2]] < Inf
        println("The marginal costs for the arc $a is: $(mc_arcs[a])")
    end
end
for a in A, p in P
    if costs[p,a[1],a[2]] < Inf
        println("The value of $(x[a,p]) is $(value(x[a,p]))")
    end
end

In [ ]:
## Raising the supply availability of product 1 at the first supply node by 1
sup[1,1] = sup[1,1] + 1;

In [ ]:
## Define model using GLPK
model = Model(GLPK.Optimizer);

## Variables
@variable(model, x[a in A, p in P; costs[p,a[1],a[2]] < Inf] >= 0);

## OF
@objective(model, Min, sum(costs[p,a[1],a[2]]*x[a,p] for p in P, a in A if costs[p,a[1],a[2]] < Inf));

## Constraints
@constraint(model, sup_con[s in S, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[1] == s) <= sup[s,p]);       # sum of everything that leaves supplier s
@constraint(model, dem_con[d in D, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[2] == d) >= dem[d,p]);       # sum of everything that arrives at demand d
@constraint(model, cap_con[a in A; cap[a[1],a[2]] < Inf], sum(x[a,p] for p in P if costs[p,a[1],a[2]] < Inf) <= cap[a[1],a[2]]);# arc capacity constraints

In [ ]:
set_silent(model)
optimize!(model)
status = termination_status(model)
println(status)

In [ ]:
## New optimal value
new_obj = objective_value(model)

In [ ]:
## Decrease in the optimal value
new_obj - obj

In [ ]:
mc_supply[1,1]

The marginal cost predicted the change in objective value correctly. Let's now try changing another bound.

In [ ]:
## Back to the original supply availability
sup[1,1] = sup[1,1] - 1;

In [ ]:
sup[1,1]

In [ ]:
## Increasing the arc capacity by 1 for the arc from supplier 2 to demand node 2
cap[2, 2] = cap[2, 2] + 1;

In [ ]:
## Define model using GLPK
model = Model(GLPK.Optimizer);

## Variables
@variable(model, x[a in A, p in P; costs[p,a[1],a[2]] < Inf] >= 0);

## OF
@objective(model, Min, sum(costs[p,a[1],a[2]]*x[a,p] for p in P, a in A if costs[p,a[1],a[2]] < Inf));

## Constraints
@constraint(model, sup_con[s in S, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[1] == s) <= sup[s,p]);       # sum of everything that leaves supplier s
@constraint(model, dem_con[d in D, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[2] == d) >= dem[d,p]);       # sum of everything that arrives at demand d
@constraint(model, cap_con[a in A; cap[a[1],a[2]] < Inf], sum(x[a,p] for p in P if costs[p,a[1],a[2]] < Inf) <= cap[a[1],a[2]]);# arc capacity constraints

In [ ]:
set_silent(model)
optimize!(model)
status = termination_status(model)
println(status)

In [ ]:
## New optimal value
new_obj = objective_value(model)

In [ ]:
new_obj-obj

In [ ]:
mc_arcs[(2, 2)]

Turns out the marginal cost that we calculated did not predict this change correctly. Comparing the solution below, we notice that one unit of product 1 to demand point 2 is now transported from supplier 2 instead of supplier 3 due to the increased capacity of arc 2->2. This shows that you should always check whether you can apply marginal costs. If the optimal basis changes because of a change in $b$, you can't apply marginal costs.

In [ ]:
## Computing the duals to infer the marginal costs
mc_supply = dual.(sup_con);
mc_arcs = dual.(cap_con);
for s in S, p in P
    println("The marginal costs for the supply $s for the product $p is: $(mc_supply[s,p])")
end
for a in A
    if cap[a[1],a[2]] < Inf
        println("The marginal costs for the arc $a is: $(mc_arcs[a])")
    end
end
for a in A, p in P
    if costs[p,a[1],a[2]] < Inf
        println("The value of $(x[a,p]) is $(value(x[a,p]))")
    end
end

### Problem 5.5 - Complementary slackness
Recall the paint factory problem introduced in Section 1.2.1. of the lecture notes

In [ ]:
A = [6 4; 1 2; -1 1; 0 1]
b = [24; 6; 1; 2]
c = [5; 4];

#### Solving the primal and dual problems
Formulate and solve the two problems to obtain their optimal solutions

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here

#### Complementary slackness
Verify the optimality of your solutions using complementary slackness

In [ ]:
# TODO: add your code here